In [42]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import pandas as pd
import time

In [20]:
driver = webdriver.Chrome()

In [43]:
# 데스노트 서울 2023
url = 'https://tickets.interpark.com/goods/23002291?app_tapbar_state=hide&'
driver.get(url)

In [34]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [44]:
close = driver.find_element(By.XPATH, '//*[@id="popup-prdGuide"]/div/div[3]/button')
close.click()

In [45]:
review = driver.find_element(By.XPATH, '//*[@id="productMainBody"]/nav/div/div/ul/li[4]/a')
review.click()

In [46]:
def review_crawller(driver_):
    
    columns = ['rate', 'review_title', 'review_text', 'review_date']
    values = []
    soup = BeautifulSoup(driver_.page_source, 'lxml')

    data_rows = soup.find_all('li', attrs={'class':'bbsItem'})[2:] # best review 중복 집계 막기 위해 처음 2개 제외
    for row in data_rows:
        blank = []
        rate = row.find('div', attrs={'class':'prdStarIcon'})
        if rate:
            rate = rate['data-star'].strip()
            blank.append(rate)
        else:
            #print("<평점 없는 리뷰 제외합니다>")
            continue
        
        review_title = row.find('strong', attrs={'class':'bbsTitleText'})
        if review_title:
            review_title = review_title.get_text()
            blank.append(review_title)
        else:
            #print("<리뷰제목 없는 리뷰 제외합니다>")
            continue

        review_text = row.find('p', attrs={'class':'bbsText'})
        if review_text:
            review_text = review_text.get_text().strip()
            review_text = review_text.replace('\r', ' ') #줄바꿈 \r 표시 공백으로 변환
            blank.append(review_text)
        else:
            #print("<리뷰내용 없는 리뷰 제외합니다>")
            continue
        
        date = row.find_all('li', attrs={'class':'bbsItemInfoList'})[1]
        if date:
            date = date.get_text()
            blank.append(date)
        else:
            #print("<작성날짜 없는 리뷰 제외합니다>")
            continue
        
        values.append(blank)
        
    df = pd.DataFrame(values, columns = columns)
    df['musical_title'] = '데스노트'
    df = df[['musical_title', 'rate', 'review_title', 'review_text', 'review_date']]
    
    return df

In [187]:
# df.review_text[1] # 짱 긴데 다 나왔음! 근데 문장 줄바꿈이 \r로 표시가 안된 텍스트는 문장끼리 이어지는 참사가..

'데스노트 작년부터 회전돌고 있어요지방사람인데도 데스노트 관극하러한달에  3~4번 서울에 갑니다이번달도 3번가고 담달에도 3번갈 예정입니다안보신분 있다면 꼭 보시라고 말씀드리고싶네요'

In [47]:
soup = BeautifulSoup(driver.page_source, 'lxml')
num_page = int(soup.find('span', {'class': 'num'}).get_text())//15
num_last_list = int(soup.find('span', {'class': 'num'}).get_text())%15

In [48]:
print(num_page)
print(num_last_list)

86
0


In [49]:
review_list = []

# general case
for k in range((num_page//10)+1):
    try:
        # 첫 번째 페이지 크롤링
        print(k*10+1)
        for j in range(15):
            more = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[4]/ul/li[{}]/div/div[2]/div[1]/a/i'.format(j+1))
            more.click()
        
        review_list.append(review_crawller(driver))

        # 2~10번째 페이지 크롤링
        for i in range(2, 11):
            print(k*10+i, '/', num_page)
            next_page = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[4]/div[2]/ol/li[{}]/a'.format(i))
            next_page.click()
            time.sleep(0.5)
            
            for j in range(15):
                try:                    
                    more = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[4]/ul/li[{}]/div/div[2]/div[1]/a/i'.format(j+1))
                    more.click()
                except:
                    pass
            
            review_list.append(review_crawller(driver))

        # 다음 열 개 페이지 로딩
        # 그런데 1-10에서 11-20으로 넘어가는 것만 좀 case가 달라서 따로 처리해줌
        if k == 0:
            next_ten = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[4]/div[2]/a')
            next_ten.click()
            time.sleep(0.5)
        else:
            next_ten = driver.find_element(By.XPATH, '//*[@id="prdReview"]/div/div[4]/div[2]/a[2]')
            next_ten.click()
            time.sleep(0.5)
    except:
        pass


2 / 86
3 / 86
4 / 86
5 / 86
6 / 86
7 / 86
8 / 86
9 / 86
10 / 86
12 / 86
13 / 86
14 / 86
15 / 86
16 / 86
17 / 86
18 / 86
19 / 86
20 / 86
22 / 86
23 / 86
24 / 86
25 / 86
26 / 86
27 / 86
28 / 86
29 / 86
30 / 86


In [41]:
len(review_list)

20

In [265]:
pd.concat(review_list).reset_index(drop=True) # index 초기화 해줘야함

,musical_title,rate,review_title,review_text,review_date
0,데스노트,5,데스노트 재밌어요,정말 재미있는 작품이에요~유일하게 회전돌고있는 뮤지컬입니다!!!,2023.05.05
1,데스노트,5,데스노트 최고입니다!!!,데스노트 작년부터 회전돌고 있어요지방사람인데도 데스노트 관극하러한달에 3~4번 서...,2023.05.05
2,데스노트,5,뮤지컬데스노트,뮤지컬데스노트 마지막 라잇홍 보세요홍광호 김성철 장지후 장은아 류인아 배우님들 다 ...,2023.05.05
3,데스노트,5,김준수 배우님 최고,데스노트 샤엘 너무 좋아요!!,2023.05.05
4,데스노트,5,최고의공연입니다,와진짜 귀호강이 이런거군요 ㅠㅠ 너무 좋아서 감동의 물결이 ㅠㅠ 최고의 공연을 봐서...,2023.05.05
...,...,...,...,...,...
895,데스노트,5,홍샤는 진리,오늘 공연..찢었습니다 홍샤는 진리예요샤엘 오늘 처음 보는데 몸 너무 잘 쓰고 홍이...,2023.04.26
896,데스노트,5,생애 첫 홍샤,후기가 좀 많이 늦긴 했지만 정말 기대를 저버리지 않은 페어였습니다 둘이서 그냥 샤...,2023.04.26
897,데스노트,5,볼때마다 짜릿한 홍샤의 데스노트,한번 본것도 아니고 여러번 보는 중인데 볼때마다각 넘버와 배우님들의 열연에 박수치고...,2023.04.26
898,데스노트,5,샤엘 샤롯데 샤랑해요~♡,여긴 어디든 꿀자리군요. 좋은 공연장에서 좋은 배우를 봐서 좋아요!!!!,2023.04.26
